# 02_anatprep
这是MRI数据pipeline的第二部分: 结构像处理
下面以 20220110_CET_CN003 为例。注意下面命令前面加！号是为了在python环境中跑bash命令

我们先设置一些variable

In [ ]:
# 设置一些variable
t1 = '20220110182949_7_t1_mpr_sag_iso_mww.nii' # set the t1 file
subj = 'CN003' # 设定一个variable
t1_2FS = f'{subj}_2FS.nii.gz' 
AFNI_HOME = os.getenv('AFNIDIR')
FREESURFER_HOME = os.getenv('FREESURFER_HOME') # 获得 FreeSurfer的主文件夹路径
MNIT1 = AFNI_HOME + '/MNI152_2009_template_SSW.nii.gz' # 我们用MNI152去掉头皮的模板作为我们的base

然后我们新建一个文件夹

In [ ]:
# 我们先走到数据文件夹
%cd /home/data/rawdata/20220110_CET_CN003
! mkdir anatpp # 新建一个文件夹用来处理t1像
! cp {t1} anatpp/ # 把t1像拷贝一份到文件夹里面
%cd anatpp/

## Step 1. 对t1像进行简单的修改处理

In [ ]:
# 把图像的中点划到统一显示空间的中点
! 3drefit -deoblique -xorigin cen -yorigin cen -zorigin cen {t1}
# 把图像的朝向划到和标准朝向一致,这一步生成t12std.nii.gz的文件
! fslreorient2std {t1} t1_2std.nii.gz 

注意以上两步仅仅只改变nifti头文件的一些信息，进行标准化，并不修改数据

In [ ]:
# 简单的剥头皮
! 3dSkullStrip -input t1_2std.nii.gz -prefix t1_2std_ss.nii.gz -orig_vol

# 上面一步把剥掉头皮的t1像和MNI模板按重心位置移动，同时移动t12std.nii.gz, 这样做是为了让其和MNI模板空间上更一致。
! @Align_Centers -cm -base {MNIT1} -dset t1_2std_ss.nii.gz -child t1_2std.nii.gz

# 清除一下
! mv t1_2std_shft.nii.gz {t1_2FS} 
! rm *t1_2std* # 删除掉多余文件

得到给FreeSurfer的T1像之后，我们拷贝一份到/home/data/subjectT1

In [ ]:
# 先检查/home/data/subjectT1/{subj} 是否存在，如果没有，创建一个文件夹
! mkdir /home/data/subjectT1/{subj}
# 把FreeSurfer的T1文件拷贝进去
! cp {t1_2FS} /home/data/subjectT1/{subj}/

这样，t1_2FS就是我们要送给FreeSurfer进行重建的t1像，往下的Step2和Step3可以平行进行

## Step 2. 送给FreeSurfer进行皮层重建

这一步可能需要5-6个小时，需要单独开启一个tmux的会话并打开ipython

然后我们来跑FreeSurfer

In [ ]:
%cd /home/data/subjectT1/{subj}
! recon-all -s {subj} -i {subj}_2FS.nii.gz -all > ./reconlog.txt

recon-all 会在/home/software/freesurfer/subjects下面创建一个{subj}名字的文件夹，例如CN003。如果该文件夹已经存在(例如recon-all中途中断，但是重新跑一次)都会报错。这时删除之前的旧文件夹就行了。所有的报错信息都在reconlog.txt这个文件夹下。

这一步可能会要很长时间。在stout上面测试跑单个被试需要6个小时。。这一步所有的输出都在reconlog.txt里面
这一步跑完之后。把非线性配准的结果文件移动到FreeSurfer被试文件夹下面

## Step 3. 将T1结构像线性和非线性配准到MNI空间

在Step2正在跑的同时，我们可以平行的开始做step3。同样的，这一步最好单开一个tmux的会话。
这一步是将个体的t1像配准到MNI标准空间和MNI152的标准模板，这是基于volume的分析的基础

In [ ]:
%cd /home/data/subjectT1/{subj}
! mkdir mni # make a new directory
! @SSwarper -input {t1_2FS} -base {MNIT1} -subid {subj} -odir mni -verb
# 创建一个剥头皮的mask，如果剥头皮效果不满意，可以手动编辑这个mask，然后再手动剥头皮
%cd mni/
! 3dcalc -a anatSS.{subj}.nii -expr 'step(a)' -prefix anatSSmask.{subj}.nii
print('Nonlinear warping done!')

这一步可能需要半小时左右，一般会早于step2结束。

## Step 4. afni中皮层加工模块SUMA的设置

切记完全成了Step2和3再来进行第四步,首先我们把step3中间的非线性配准的文件夹放到freesurfer里面去

In [ ]:
! mv /home/data/subjectT1/{subj}/mni {FREESURFER_HOME}/subjects/{subj}/ # 移动step3获得到mni文件夹到step2生成的CN003文件夹

AFNI中用来进行皮层加工模块的是SUMA。利用FreeSurfer进行了皮层重建之后，需要对SUMA进行一定的设置

In [ ]:
! @SUMA_Make_Spec_FS -sid {subj} -fspath {FREESURFER_HOME}/subjects/{subj} -NIFTI > SUMA_Make_Spec_FS.log

以上的命令会在FreeSurfer subjects中的例如CN003的文件夹下再创建一个SUMA的文件夹，里面包含了SUMA需要的一些文件。SUMA是连接AFNI和FreeSurfer之间的工具，让在AFNI上进行皮层分析成为可能。

这一步可能需要10分钟左右。完成之后，我们打开远程桌面，打开一个terminal (注意这一步是bash命令，不用ipython)，输入以下的命令，可以打开afni和suma来检查互相的联通情况。

In [ ]:
# 注意替换被试名
afni -niml & suma -spec $FREESURFER_HOME/subjects/$subj/SUMA/CN003_both.spec \
-sv $FREESURFER_HOME/subjects/$subj/SUMA/CN003_SurfVol.nii

afni -niml & suma -spec CN003_both.spec -sv CN003_SurfVol.nii

## 总结

下面总结一下结构像处理的要点
* 结构像一般只需要处理t1像
* 在跑FreeSurfer之前，最好通过3drefit, @Align_Centers等函数让被试的t1像和MNI152的t1模板在空间位置上尽可能的重合。有时候可能需要手动调整。这一步需要利用fsleyes来人工确认。在跑FreeSurfer之前。
* 送给FreeSurfer之前的t1像，在stout上面通常会备份在例如/home/data/subjectT1/CN003.FreeSurfer皮层重建的信息也会保存在这
* 在FreeSurfer进行皮层重建，会在FreeSurfer的被试文件夹下面，在stout上面就是/home/software/freesurfer/subjects 创建一个以被试名字命名的文件夹
* 利用afni做基于volume的非线性配准，会生成一个mni的文件夹,最后会copy到freesurfer下面被试的文件夹
* 利用afni做基于volume的非线性配准，完成只有需要人工检查配准结果
* 利用afni对FreeSurfer的结构做重新整理，freesurfer下面该被试的文件夹会生成一个SUMA的文件夹